In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
import re
import folium
from folium.plugins import MarkerCluster
import nltk

# Load the data
path = '/Users/rfd/vigilo-notebooks-bapav/vigilo2024/data_collection'
df_incidents = pd.read_csv(f'{path}/df_incidents.csv')
df_poi = pd.read_csv(f'{path}/df_poi.csv')
df_incidents.head()

,token,latitude,longitude,address,comment,explanation,time,categorie,cityname
0,AA1CA063,43.606022,3.873078,"Cours Gambetta, Montpellier",Roue dans les Rails du tram,La personne venait d’une rue perpendiculaire e...,2024-05-16 11:55:00,9,Montpellier
1,AF00A88E,43.617920,3.882070,"Rue de la Cavalerie, Montpellier",stationnement gênant récurrent,stationnement gênant sur le trottoir systémati...,2024-05-15 19:03:00,2,Montpellier
2,B9B3C9BA,43.596166,3.897689,"Rue des Acconiers, Montpellier",Risque de chute,Poteau anti-stationnement en travers de la piste,2024-05-11 17:32:00,2,Montpellier
3,1441837B,43.598695,3.904834,"Avenue Nina Simone, Montpellier",Road to nowhere,NaN,2024-05-08 15:34:00,3,Montpellier
4,CA159445,43.696450,3.810200,"Rue de Coulondres, Saint-Gély-du-Fesc",Véhicule stationné sur piste cyclable,NaN,2024-05-08 09:29:00,2,Saint-Gély-du-Fesc


In [2]:
#Filtering the comment and explanation columns by a list of keywords

#List of keywords
keywords = ['arceau', 'arceaux']

#Filtering the comment and explanation columns by the list of keywords
df_incidents['comment'] = df_incidents['comment'].str.lower()
df_incidents['explanation'] = df_incidents['explanation'].str.lower()
df_incidents['keywords'] = df_incidents['comment'] + df_incidents['explanation']
df_incidents['keywords'] = df_incidents['keywords'].str.contains('|'.join(keywords))
df_incidents = df_incidents[df_incidents['keywords'] == True]
df_incidents = df_incidents.drop(columns=['keywords'])
df_incidents.head()

,token,latitude,longitude,address,comment,explanation,time,categorie,cityname
70,F1B28E6B,43.612869,3.839378,"Avenue des Garrats, Montpellier",arceaux sciés place vélos géant casino celleneu,"découverte que des arceaux avaient été sciés, ...",2023-12-11 15:05:00,100,Montpellier
340,A3E972AD,43.611758,3.865761,"Boulevard des Arceaux, Montpellier",poids lourd garé sur la piste cyclable,garé sans scrupule en plein sur la piste cycla...,2023-07-11 10:45:00,7,Montpellier
391,1802094B,43.615331,3.863613,"avenue d’Assas, Montpellier",véhicule garé empiétant sur la bande cyclable,un classique de l’avenue des arceaux avec ses ...,2023-07-03 06:36:00,2,Montpellier
996,167E906C,43.611638,3.863639,"Boulevard des Arceaux, Montpellier",vol vélo orbea vibe h30 eq. taille l bleu.,on m'a volé mon vélo le mercredi 05 avril 2023...,2023-04-05 20:00:00,10,Montpellier
1156,FFB78D63,43.617620,3.824360,"Rue Maurice Béjart, Montpellier",parc 2000: manque total des aménagements cyclable,parc 2000/pôle santé : un zone dominé par la v...,2023-03-09 09:56:00,8,Montpellier
